### Dependencies
___

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType

### Create Session
____

In [2]:
csv_file = "/tmp/walking/dataset.csv"

spark = SparkSession.builder.appName('sampleApplication').getOrCreate()

#### Create DataFrame
______

In [4]:
df = spark.read.csv(path=csv_file,header=True)
df.printSchema()

root
 |-- time: string (nullable = true)
 |-- avg_rss12: string (nullable = true)
 |-- var_rss12: string (nullable = true)
 |-- avg_rss13: string (nullable = true)
 |-- var_rss13: string (nullable = true)
 |-- avg_rss23: string (nullable = true)
 |-- var_rss23: string (nullable = true)



In [5]:
df.show(10)

+----+---------+---------+---------+---------+---------+---------+
|time|avg_rss12|var_rss12|avg_rss13|var_rss13|avg_rss23|var_rss23|
+----+---------+---------+---------+---------+---------+---------+
|   0|    35.00|     3.67|    16.50|     3.77|    14.00|     1.63|
| 250|    28.50|     3.35|    17.50|     3.77|    12.25|     3.90|
| 500|    35.50|     2.87|    15.75|     2.86|    17.75|     5.07|
| 750|    29.75|    12.19|    16.25|     2.17|    20.75|     2.59|
|1000|    27.00|     2.12|    16.75|     0.83|    24.75|     1.64|
|1250|    36.00|     5.61|    12.25|     2.28|    19.25|     2.68|
|1500|    38.50|     4.33|    16.00|     5.70|    15.25|     2.38|
|1750|    35.25|     2.49|    16.00|     1.41|    15.00|     2.83|
|2000|    35.75|     7.05|    11.00|     4.42|    12.50|     8.08|
|2250|    36.00|     7.04|    15.00|     2.12|    16.75|     4.02|
+----+---------+---------+---------+---------+---------+---------+
only showing top 10 rows



In [6]:
df.describe().show()

+-------+----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|            time|        avg_rss12|         var_rss12|         avg_rss13|         var_rss13|         avg_rss23|         var_rss23|
+-------+----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|            7200|             7200|              7200|              7200|              7200|              7200|              7200|
|   mean|         59875.0|34.43367638888891| 4.337393055555574|15.394559722222217|3.2036083333333623|16.025058333333345| 3.343612500000015|
| stddev|34643.3468469953|4.808602849860674|2.4434541249050907| 2.912519317367833| 1.624393703577679|3.1124831754792432|1.7012372763482624|
|    min|               0|            12.50|              0.00|             10.00|              0.00|             10.00|              0.00|
|    max|           

#### Schema Casting
_____

In [7]:
data_schema = [StructField('time', IntegerType(), False),
               StructField('avg_rss12', FloatType(), False),
               StructField('var_rss12', FloatType(), False),
               StructField('avg_rss13', FloatType(), False),
               StructField('var_rss13', FloatType(), False),
               StructField('avg_rss23', FloatType(), False),
               StructField('var_rss23', FloatType(), False),
              ]

final_struct = StructType(fields=data_schema)

df = spark.read.csv(path=csv_file, header=True, schema=final_struct)
df.printSchema()

root
 |-- time: integer (nullable = true)
 |-- avg_rss12: float (nullable = true)
 |-- var_rss12: float (nullable = true)
 |-- avg_rss13: float (nullable = true)
 |-- var_rss13: float (nullable = true)
 |-- avg_rss23: float (nullable = true)
 |-- var_rss23: float (nullable = true)



#### Selecting Data
_____

In [10]:
# pyspark.sql.column.Column 
# returns only the column object
type(df['time'])


pyspark.sql.column.Column

In [13]:
# selecting the column data and returns a dataframe
df.select('time').show()

+----+
|time|
+----+
|   0|
| 250|
| 500|
| 750|
|1000|
|1250|
|1500|
|1750|
|2000|
|2250|
|2500|
|2750|
|3000|
|3250|
|3500|
|3750|
|4000|
|4250|
|4500|
|4750|
+----+
only showing top 20 rows



In [15]:
# head method

df.head(2)

[Row(time=0, avg_rss12=35.0, var_rss12=3.6700000762939453, avg_rss13=16.5, var_rss13=3.7699999809265137, avg_rss23=14.0, var_rss23=1.6299999952316284),
 Row(time=250, avg_rss12=28.5, var_rss12=3.3499999046325684, avg_rss13=17.5, var_rss13=3.7699999809265137, avg_rss23=12.25, var_rss23=3.9000000953674316)]

In [17]:
df.head(1)[0]

Row(time=0, avg_rss12=35.0, var_rss12=3.6700000762939453, avg_rss13=16.5, var_rss13=3.7699999809265137, avg_rss23=14.0, var_rss23=1.6299999952316284)

#### Working with SQL
_____

In [27]:
df.createOrReplaceTempView('measurements')

results = spark.sql("SELECT * FROM measurements WHERE time=250")

type(results)

pyspark.sql.dataframe.DataFrame

In [28]:
results.show()

+----+---------+---------+---------+---------+---------+---------+
|time|avg_rss12|var_rss12|avg_rss13|var_rss13|avg_rss23|var_rss23|
+----+---------+---------+---------+---------+---------+---------+
| 250|     28.5|     3.35|     17.5|     3.77|    12.25|      3.9|
| 250|     40.0|     5.24|    19.25|     2.86|    12.25|     4.32|
| 250|    36.25|     7.26|     15.0|     3.67|    17.75|     2.86|
| 250|    33.67|     3.86|    13.33|      3.4|     16.0|      5.0|
| 250|     40.0|     4.06|     12.0|     3.94|     13.0|     7.38|
| 250|    31.75|     2.49|    16.25|     5.31|     17.0|     2.24|
| 250|     37.5|     1.12|    16.75|      4.6|    17.25|     4.32|
| 250|    40.25|     2.49|    12.75|     4.32|    23.25|      1.3|
| 250|    37.75|     5.07|     15.5|      3.2|    14.75|     1.79|
| 250|     39.0|      3.0|     21.0|      1.0|     17.0|     5.79|
| 250|    30.25|     6.94|    22.25|     3.03|    16.75|     5.49|
| 250|    41.25|     4.44|    17.33|      4.5|    14.67|     3